# Setup Notebook
Common setup for data engineering workflows


In [ ]:
# Import framework to verify it's available
from clinical_data_standards_framework import utils
from pyspark.sql import functions as F
import time
print("✅ Framework loaded successfully")


In [ ]:
# Get parameters from job
flow_name = dbutils.widgets.get("flow_name")
databricks_run_id = dbutils.widgets.get("databricks_run_id")
databricks_job_id = dbutils.widgets.get("databricks_job_id")
databricks_job_name = dbutils.widgets.get("databricks_job_name")
created_by_principal = dbutils.widgets.get("created_by_principal")
workspace_file_path = dbutils.widgets.get("workspace_file_path")

print(f"Flow: {flow_name}")
print(f"Job ID: {databricks_job_id}")
print(f"Job Name: {databricks_job_name}")
print(f"Run ID: {databricks_run_id}")
print(f"Created By: {created_by_principal}")

import json

start_time = time.time()

# ============================================================================
# Get catalog from catalog_override parameter (REQUIRED)
# This is the single source of truth for catalog - passed from job parameters
# Default: ${var.catalog} from databricks.yml
# ============================================================================
try:
    catalog_override = dbutils.widgets.get("catalog_override")
    if catalog_override and catalog_override.strip():
        default_catalog = catalog_override
        print(f"📦 Using catalog: {default_catalog}")
    else:
        raise ValueError("catalog_override parameter is empty")
except Exception as e:
    raise RuntimeError(f"catalog_override parameter required. Error: {e}")

default_bronze_schema = "bronze_md"
cache_table = f"{default_catalog}.{default_bronze_schema}.md_config_cache"

# ============================================================================
# Read configuration from Delta cache table (md_config_cache)
# Pre-requisite: Run job_cds_sql_load_config_data to populate the cache table
# ============================================================================
print(f"📋 Reading config from: {cache_table}")

config_df = spark.table(cache_table).filter(
    F.col("config_key") == "clinical_data_standards"
).select("config_section", "config_json")

# Load all sections from cache
configs = {row.config_section: json.loads(row.config_json) for row in config_df.collect()}

if not configs:
    raise RuntimeError(f"Config cache table {cache_table} is empty. Run job_cds_sql_load_config_data first.")

globals_dict = configs.get("globals", {})
services_dict = configs.get("services", {})
versioning_dict = configs.get("versioning", {})
streaming_dict = configs.get("streaming", {})
pipelines_dict = configs.get("pipelines", {})

# Always update globals with the active catalog from catalog_override
original_catalog = globals_dict.get('catalog')
if original_catalog != default_catalog:
    globals_dict['catalog'] = default_catalog
    print(f"\n🔄 Catalog updated in globals:")
    print(f"  Config default: {original_catalog}")
    print(f"  Active catalog: {default_catalog}")

# Get pipeline config for this flow
pipeline_config = pipelines_dict.get(flow_name) if flow_name and pipelines_dict else None

elapsed = time.time() - start_time
print(f"\n⚡ Config loaded from Delta cache in {elapsed:.2f}s")

print(f"\n✅ Config loaded:")
print(f"  - Globals: {list(globals_dict.keys())}")
print(f"  - Services: {list(services_dict.keys())}")
print(f"  - Versioning: {list(versioning_dict.keys()) if versioning_dict else 'Not configured'}")
print(f"  - Streaming: {list(streaming_dict.keys()) if streaming_dict else 'Not configured'}")
print(f"  - Pipeline: {flow_name}")

# Display key globals
print(f"\n📦 Globals:")
print(f"  Catalog: {globals_dict.get('catalog')}")
print(f"  Bronze Schema: {globals_dict.get('bronze_schema')}")
print(f"  Silver Schema: {globals_dict.get('silver_schema')}")
print(f"  Gold Schema: {globals_dict.get('gold_schema')}")

# Set task values for downstream tasks - Job tracking
dbutils.jobs.taskValues.set(key="flow_name", value=flow_name)
dbutils.jobs.taskValues.set(key="databricks_run_id", value=databricks_run_id)
dbutils.jobs.taskValues.set(key="databricks_job_id", value=databricks_job_id)
dbutils.jobs.taskValues.set(key="databricks_job_name", value=databricks_job_name)
dbutils.jobs.taskValues.set(key="created_by_principal", value=created_by_principal)

# Set task values for downstream tasks - Config as JSON strings
dbutils.jobs.taskValues.set(key="globals", value=json.dumps(globals_dict))
dbutils.jobs.taskValues.set(key="services", value=json.dumps(services_dict))
dbutils.jobs.taskValues.set(key="versioning", value=json.dumps(versioning_dict))
dbutils.jobs.taskValues.set(key="streaming_config", value=json.dumps(streaming_dict))
if pipeline_config:
    dbutils.jobs.taskValues.set(key="pipeline_config", value=json.dumps(pipeline_config))

print("\n✅ Task values set for downstream tasks")
print("   Downstream notebooks can access via:")
print("   globals = json.loads(dbutils.jobs.taskValues.get(taskKey='setup', key='globals'))")
print("   services = json.loads(dbutils.jobs.taskValues.get(taskKey='setup', key='services'))")
print("   versioning = json.loads(dbutils.jobs.taskValues.get(taskKey='setup', key='versioning'))")
print("   streaming_config = json.loads(dbutils.jobs.taskValues.get(taskKey='setup', key='streaming_config'))")
print("   pipeline_config = json.loads(dbutils.jobs.taskValues.get(taskKey='setup', key='pipeline_config'))")

# ================================
# Handle Path Overrides for File Processor
# ================================
# Read optional workflow parameters (widgets defined in job base_parameters)
# dbutils.widgets.get() only takes 1 argument
try:
    source_root_param = dbutils.widgets.get("source_root")
    source_subdir_param = dbutils.widgets.get("source_subdir")
    target_subdir_param = dbutils.widgets.get("target_subdir")
    
    # Compute source/target paths
    if source_subdir_param:
        # Full path override provided
        source_subdir = source_subdir_param
        target_subdir = target_subdir_param if target_subdir_param else source_subdir_param.replace('/uploads', '/ingested')
        print(f"\n🔧 Using workflow parameter override:")
        print(f"  Source: {source_subdir}")
        print(f"  Target: {target_subdir}")
    else:
        # Compute from root (or use default)
        if source_root_param:
            source_root = source_root_param
        else:
            # Get default from config
            source_root = pipeline_config.get('default_source', 'historical_data') if pipeline_config else 'historical_data'
        
        # Compute: {root}/uploads → {root}/ingested
        source_subdir = f"{source_root}/uploads"
        target_subdir = f"{source_root}/ingested"
        print(f"\n📁 Using config default (root: {source_root}):")
        print(f"  Source: {source_subdir}")
        print(f"  Target: {target_subdir}")
    
    # Also get volume names from config
    source_volume = pipeline_config.get('source_volume', 'clinical_data_standards') if pipeline_config else 'clinical_data_standards'
    target_volume = pipeline_config.get('target_volume', 'clinical_data_standards') if pipeline_config else 'clinical_data_standards'
    
    # Set task values for file processor to use
    dbutils.jobs.taskValues.set(key="source_volume", value=source_volume)
    dbutils.jobs.taskValues.set(key="source_subdir", value=source_subdir)
    dbutils.jobs.taskValues.set(key="target_volume", value=target_volume)
    dbutils.jobs.taskValues.set(key="target_subdir", value=target_subdir)
    
    print(f"\n✅ Path configuration set for file processor")
except Exception:
    # These widgets don't exist for non-file-processor jobs - silently skip
    pass
